# Mixtures of MDPs -- Actual Figures

In [ ]:
# Imports 


import sys
# setting path
sys.path.append('../core')
import os
#os.environ["VECLIB_MAXIMUM_THREADS"] = "10"

from sklearn.decomposition import PCA

#bread and butter
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

#machine learning library
import sklearn
import sklearn.manifold
import sklearn.cluster
import sklearn.decomposition

#statistics
import scipy
from scipy.stats import rankdata, norm

import numba
from numba import jit, njit

from tqdm import tqdm
import copy
import cvxpy as cp

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
#import numpy as tnp
import warnings
warnings.filterwarnings('ignore')

# import dfply
# from dfply import *
# import ray
# import datetime


import os
import warnings
warnings.filterwarnings("ignore")

#import torch
#torch.cuda.is_available()
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
import scipy

import confound_mdp
import confound_ope
import confound_env
import copy
import pickle

from subspace import *
from clustering import *
from emalg import *
from helpers import *


import multiprocessing

%matplotlib inline
plt.style.use('matplotlibrc')

In [ ]:
#FLIP THIS TO TRUE IF YOU WANT TO SEE MORE PLOTS WHILE THINGS ARE RUNNING
diagnostic = False

horizons = [20, 30, 40, 50, 60, 70, 80, 90, 100]
#horizons = [100]

#BY DEFAULT SET TO 30
trials = 30



clusterAccsTrials = []
clusterAccsNoProjsTrials = []
clusterAccsRandProjsTrials = []
classAccsTrials = []
classProjAccsTrials = []
hardClustEMAccsTrials = []
softClustEMAccsTrials = []
hardClassEMAccsTrials = []
softClassEMAccsTrials = []
hardEMAccsTrials = []
softEMAccsTrials = []

for trial in range(trials):
    clusterAccs = []
    clusterAccsNoProjs = []
    clusterAccsRandProjs = []
    modelEstims = []
    classAccs = []
    classProjAccs = []
    hardClustEMs = []
    hardClustEMAccs = []
    softClustEMs = []
    softClustEMAccs = []
    softClassEMs = []
    softClassEMAccs = []
    hardClassEMs = []
    hardClassEMAccs = []
    hardEMs = []
    hardEMAccs = []
    softEMs = []
    softEMAccs = []
    
    start_seed = trial*multiprocessing.cpu_count()
    print("Trial", trial, "with start seed", start_seed)

    for horizon in horizons:
        print("Current Horizon is:", horizon)

        ###GRIDWORLD###
    
        pi_b, P, R, x_dist, u_dist, gamma = confound_env.gridworld_opetools(horizon = horizon, slip = 0.04, 
                                                                        confound_weight=0.6, small=False, soft=True)
        pi_b[0] = pi_b.mean(0)
        pi_b[1] = pi_b.mean(0)

        mdp = confound_mdp.ConfoundMDP(P, R, x_dist, u_dist, gamma)

        nStates = P.shape[2]
        nActions = P.shape[1]

        ###GRIDWORLD###

        dataset = getSamplesMultiProc(1000, mdp, pi_b, horizon, start_seed=start_seed, iid=False)

         # behavior value
        print("value of pi_b")
        returns = confound_mdp.calc_returns(dataset, gamma, horizon)
        print(returns.mean())
    

        stateactions = dataset[:, :, [0, 1]]
        memorder = 'C'

        if memorder == 'F':
            stateactions = (dataset[:,:,0] + dataset[:,:,1]*nStates).astype(int) #we code (S x A) as s + a|S|, 0,16,32,48 1,17,33,49
        else:
            stateactions = (dataset[:,:,0]*nActions + dataset[:,:,1]).astype(int)
        states = (dataset[:,:,0]).astype(int)
        actions = (dataset[:,:,1]).astype(int)
        currstates = dataset[:,:,[0,1]].astype(int)
        confounders = dataset[:,:,2].astype(int)
        labels = confounders[:,0].astype(int)
        nextstates = dataset[:,:,3].astype(int)

        L = 2
        K = 2
        #S = nStates*nActions
        #N = len(threes)


        ########SUBSPACE ESTIMATION########

        sadim = nStates*nActions
        spdim = nStates
        omegaone = np.array([i for i in range(int(horizon/4), 2*int(horizon/4))])
        omegatwo = np.array([i for i in range(3*int(horizon/4), horizon)])
        onehotsa = np.eye(sadim)[stateactions]
        #onehotsaflat = copy.deepcopy(onehotsa)
        onehotsa = onehotsa.reshape((onehotsa.shape[0], onehotsa.shape[1], nStates, nActions), order=memorder)
        onehotsp = np.eye(np.max(nextstates)+1)[nextstates]
        sz = int(onehotsa.shape[0]/3)

        eigvalsa, eigvecsa = getEig(onehotsa[:sz], onehotsp[:sz], omegaone, omegatwo, K, wt=True, smalldata=False, device='/GPU:0')

        #########CLUSTERING#########

        device = "/GPU:0"

        hs = np.array([geth(onehotsa[sz:,omegaone,:,:], 
                                         onehotsp[sz:,omegaone,:]), 
                                  geth(onehotsa[sz:,omegatwo,:,:], 
                                       onehotsp[sz:,omegatwo,:])], dtype=np.float32)

        statmns = computeStat(hs, 
                          eigvecsa, numpy=False, smalldata=False, device=device, proj=True)

        statNoProjs = computeStat(hs, 
                          eigvecsa, numpy=False, smalldata=False, device=device, proj=False)
        
        randProjs = np.linalg.qr(np.random.normal(size=(nStates,nActions,nStates,K)))[0]
        statRandProjs = computeStat(hs, 
                                    randProjs, numpy=False, smalldata=False, device=device, proj=True)
        
        if diagnostic:
            clusterDiagnostics(statmns, K=K, labels=confounders[sz:, 0],
                           lo=0, hi=0.0005, step=0.000025) #0,0.01, 0.0001
            plt.show()
            clusterDiagnostics(statNoProjs, K=K, labels=confounders[sz:, 0],
                       lo=0, hi=0.0005, step=0.000025) #0,0.01, 0.0001
            plt.show()
            clusterDiagnostics(statRandProjs, K=K, labels=confounders[sz:, 0],
                       lo=0, hi=0.0005, step=0.000025) #0,0.01, 0.0001
            plt.show()

        clusterlabs = sklearn.cluster.spectral_clustering((statmns < 0.0001).astype(int), n_clusters=2,
                                                         assign_labels='kmeans')

        clusterlabsNoProjs = sklearn.cluster.spectral_clustering((statNoProjs < 0.0001).astype(int), n_clusters=2,
                                                         assign_labels='kmeans')
        
        clusterlabsRandProjs = sklearn.cluster.spectral_clustering((statRandProjs < 0.0001).astype(int), n_clusters=2,
                                                         assign_labels='kmeans')
    
        
        
        
        permutation = np.argmax([np.mean(clusterlabs == confounders[sz:, 0]), 
                        np.mean(clusterlabs != confounders[sz:, 0])])
        if diagnostic:
            print(permutation, np.max([np.mean(clusterlabs == confounders[sz:, 0]), 
                        np.mean(clusterlabs != confounders[sz:, 0])]))
        clusterAccs.append(np.max([np.mean(clusterlabs == confounders[sz:, 0]), 
                        np.mean(clusterlabs != confounders[sz:, 0])]))

        clusterAccsNoProjs.append(np.max([np.mean(clusterlabsNoProjs == confounders[sz:, 0]), 
                        np.mean(clusterlabsNoProjs != confounders[sz:, 0])]))
        
        clusterAccsRandProjs.append(np.max([np.mean(clusterlabsRandProjs == confounders[sz:, 0]), 
                        np.mean(clusterlabsRandProjs != confounders[sz:, 0])]))

        ########MODEL ESTIMATION#########
        Phat_ksa = getModelEstim(clusterlabs.astype(int), 
                             states[sz:,:], 
                             actions[sz:,:], 
                             nextstates[sz:,:],
                            K=K, nStates=nStates, nActions=nActions, hard=True)
        modelEstims.append(Phat_ksa)


        ########CLASSIFICATION#########
        priorclass = np.bincount(clusterlabs)/len(clusterlabs) 
        piclust = getPolicyEstim(states[sz:], actions[sz:], 
                                 K, nStates, nActions, preds=clusterlabs, hard=True)
        startweights = getStartWeights(states[sz:], clusterlabs, K, nStates)
        subspaceClass = classify(Phat_ksa, states[:sz,:], actions[:sz,:], 
                               nextstates[:sz,:], piclust, reg=1, 
                           prior=priorclass, startweights = startweights, labs=True)
        if diagnostic:
            print('Subspace Classification accuracy:', [np.mean(labels[:sz] == subspaceClass), 
                                                    np.mean(labels[:sz] != subspaceClass)][permutation])
        classAccs.append(np.max([np.mean(labels[:sz] == subspaceClass), np.mean(labels[:sz] != subspaceClass)]))
        
        hsubs = np.array([geth(onehotsa[:sz,omegaone,:,:], 
                                         onehotsp[:sz,omegaone,:]), 
                                  geth(onehotsa[:sz,omegatwo,:,:], 
                                       onehotsp[:sz,omegatwo,:])], 
                 dtype=np.float32)
        projclass = classifyProj(dataset[sz:], clusterlabs, hsubs, Phat_ksa, K, nStates, nActions)
        classProjAccs.append(np.max([np.mean(labels[:sz] == projclass), np.mean(labels[:sz] != projclass)]))


        ########EM ALGORITHM########
        priorclass = np.bincount(clusterlabs)/len(clusterlabs) 
        piclust = getPolicyEstim(states[sz:], actions[sz:], 
                                 K, nStates, nActions, preds=clusterlabs, hard=True)

        startweights = getStartWeights(states[sz:], clusterlabs, K, nStates)
        maxapos = classify(Phat_ksa, states, actions, 
                               nextstates, piclust, reg=1, 
                           prior=priorclass, startweights = startweights, labs=True)
        mleprobs = classify(Phat_ksa, states, actions, 
                               nextstates, piclust, reg=1, 
                            prior=priorclass, startweights = startweights, labs=False)
        if diagnostic:
            print('MAP estimate overall accuracy:', 
                  [np.mean(labels == maxapos), np.mean(labels != maxapos)][permutation])

        # HARD EM 
        expectclass, modelestimclass, loglikclass = em(maxapos, 
                                Phat_ksa, states, actions, 
                            nextstates, labels=labels, 
                            K=K, nStates=nStates, nActions=nActions,
                            prior = priorclass, reg = 1, max_iter=100,
                                                   permute=False, permutation=permutation,
                                                   checkin=1, hard=True, verbose=False)
        hardClustEMs.append([expectclass, modelestimclass, loglikclass])
        hardClustEMAccs.append(np.max([np.mean(labels == expectclass), np.mean(labels != expectclass)]))

        # SOFT EM MODEL INIT
        normprobs = np.exp(mleprobs-np.max(mleprobs))
        regprobs = 0.8*(normprobs)/np.nansum(np.abs(normprobs), 0) + 0.1*np.ones(mleprobs.shape)
        expectclasssoft, modelestimclasssoft, loglikclasssoft = em(regprobs, 
                                    Phat_ksa, states, actions, 
                                nextstates, labels, 
                                K=2, nStates=nStates, nActions=nActions,
                                prior = priorclass, reg = 1, permute=False,
                                permutation=permutation, checkin=1, hard=False, verbose=False)
        softClustEMs.append([expectclasssoft, modelestimclasssoft, loglikclasssoft])
        softClustEMAccs.append(np.max([np.mean(labels == expectclasssoft.argmax(0)),
                                np.mean(labels != expectclasssoft.argmax(0))]))
        
        #HARD EM CLASS LABEL INIT
        expecthard, modelestimhard, loglikhard = em(np.hstack([projclass, clusterlabs]), 
                                    Phat_ksa, states, actions, 
                                nextstates, labels, 
                                K=2, nStates=nStates, nActions=nActions,
                                prior = priorclass, reg = 1, permute=False,
                                permutation=permutation, checkin=1, hard=True, verbose=False)
        hardClassEMs.append([expecthard, modelestimhard, loglikhard])
        hardClassEMAccs.append(np.max([np.mean(labels == expecthard.argmax(0)),
                                np.mean(labels != expecthard.argmax(0))]))
        
        #SOFT EM CLASS LABEL INIT
        expect, modelestim, loglik = em(0.8*np.eye(K)[np.hstack([projclass, clusterlabs])].T +
                                        0.1*np.ones((K, len(np.hstack([projclass, clusterlabs])))), 
                                    Phat_ksa, states, actions, 
                                nextstates, labels, 
                                K=2, nStates=nStates, nActions=nActions,
                                prior = priorclass, reg = 1, permute=False,
                                permutation=permutation, checkin=1, hard=False, verbose=False)
        softClassEMs.append([expect, modelestim, loglik])
        softClassEMAccs.append(np.max([np.mean(labels == expect.argmax(0)),
                                np.mean(labels != expect.argmax(0))]))

        #############BASE EM###############

        labelsem = []
        logliksem = []
        for i in tqdm(range(30)):
            randlabs = np.random.binomial(1, 0.5, size=len(dataset)).astype(int)
            randmodel = getModelEstim(randlabs, states, actions, nextstates,
                                      K, nStates, nActions, hard=True)
            randlabs, randmodel, loglikrand = em(randlabs, randmodel,
               states, actions, nextstates, labels, 
                                K=K, nStates=nStates, nActions=nActions,
                                prior = priorclass, reg = 0, permute=True, permutation=permutation, checkin=1, verbose=False, hard=True)
            labelsem.append(randlabs)
            logliksem.append(loglikrand)
        if diagnostic:
            plt.figure(figsize=(12,7))
            plt.scatter([max(np.mean(i == labels), np.mean(i != labels)) for i in labelsem], logliksem)
            print(np.mean([max(np.mean(i == labels), np.mean(i != labels)) for i in labelsem]))
            print(np.median([max(np.mean(i == labels), np.mean(i != labels)) for i in labelsem]))
            print(np.max([max(np.mean(i == labels), np.mean(i != labels)) for i in labelsem]))
            plt.xlabel('Accuracy')
            plt.ylabel('Log-likelihood')
            plt.title('Randomly Initialized Hard EM Algorithm')
            plt.show()
            plt.hist([max(np.mean(i == labels), np.mean(i != labels)) for i in labelsem])
            plt.show()
        hardEMAccs.append(np.mean([max(np.mean(i == labels), np.mean(i != labels)) for i in labelsem]))
        hardEMs.append([labelsem, logliksem])

        labelsemsoft = []
        logliksemsoft = []
        for i in tqdm(range(30)):
            unifs = np.random.uniform(size=len(dataset))
            randlabsoft = np.vstack([unifs, 1-unifs])
            randmodelsoft = getModelEstim(randlabsoft, states, actions, nextstates,
                                      K, nStates, nActions, hard=False)
            randlabsoft, randmodelsoft, loglikrand = em(randlabsoft, randmodelsoft,
           states, actions, nextstates, labels, 
                                K=K, nStates=nStates, nActions=nActions,
                                prior = priorclass, reg = 0, permute=True, permutation=permutation, checkin=1, verbose=False, hard=False)
            labelsemsoft.append(randlabsoft)
            logliksemsoft.append(loglikrand)
        if diagnostic:
            plt.figure(figsize=(12,7))
            plt.scatter([max(np.mean(i == labels), np.mean(i != labels)) for i in np.array(labelsemsoft).argmax(axis=1)], 
                        logliksemsoft)
            print(np.mean([max(np.mean(i == labels), np.mean(i != labels)) for i in np.array(labelsemsoft).argmax(axis=1)]))
            print(np.median([max(np.mean(i == labels), np.mean(i != labels)) for i in np.array(labelsemsoft).argmax(axis=1)]))
            print(np.max([max(np.mean(i == labels), np.mean(i != labels)) for i in np.array(labelsemsoft).argmax(axis=1)]))
            plt.ticklabel_format(style='plain',useOffset=False)
            plt.xlabel('Accuracy')
            plt.ylabel('Log-likelihood')
            plt.title('Randomly Initialized Soft EM Algorithm')
            plt.show()
            plt.hist([max(np.mean(i == labels), np.mean(i != labels)) for i in np.array(labelsemsoft).argmax(axis=1)])
            plt.show()
        softEMAccs.append(np.mean([max(np.mean(i == labels), np.mean(i != labels)) for i in np.array(labelsemsoft).argmax(axis=1)]))
        softEMs.append([labelsemsoft, logliksemsoft])
        
    clusterAccsTrials.append(clusterAccs)
    clusterAccsNoProjsTrials.append(clusterAccsNoProjs)
    clusterAccsRandProjsTrials.append(clusterAccsRandProjs)
    classAccsTrials.append(classAccs)
    classProjAccsTrials.append(classProjAccs)
    hardClustEMAccsTrials.append(hardClustEMAccs)
    softClustEMAccsTrials.append(softClustEMAccs)
    hardEMAccsTrials.append(hardEMAccs)
    softEMAccsTrials.append(softEMAccs)
    hardClassEMAccsTrials.append(hardClassEMAccs)
    softClassEMAccsTrials.append(softClassEMAccs)

In [ ]:
randAccs = []
dims = [2,4,6,8,12,16,24,32,40,48,56,64,72,80]
for dim in dims:
    randProjs = np.linalg.qr(np.random.normal(size=(nStates,nActions,nStates,dim)))[0]
    statRandProjs = computeStat(hs, 
                            randProjs, numpy=False, smalldata=False, device=device, proj=True)
    clusterRands = sklearn.cluster.spectral_clustering((statRandProjs < 0.0001).astype(int), n_clusters=2,
                                                         assign_labels='kmeans')
    randAccs.append(np.max([np.mean(clusterRands == confounders[sz:, 0]), 
                        np.mean(clusterRands != confounders[sz:, 0])]))
        
#clusterDiagnostics(statRandProjs, K=K, labels=confounders[sz:, 0],
#               lo=0, hi=0.0005, step=0.000025) #0,0.01, 0.0001
#plt.show()
plt.plot(dims, 1-np.array(randAccs), label='Random Projections')
plt.axhline(1-clusterAccs[-1], color='darkred', linestyle='dotted', label='Subspace Projections')
plt.axhline(1-clusterAccsNoProjs[-1], color='orange', linestyle='dashed', label='No Projections')
plt.xlabel('Dimension of Random Embedding')
plt.ylabel('Clustering Error')
plt.title('Clustering Error v.s. Random Embedding Dimension')
plt.legend()
plt.savefig('figs/jlRandEmbedding.png')

In [ ]:
import seaborn as sns
#plt.figure(figsize=(16,9))
plt.hist(statmns.flatten(), bins=200, density=True)#[1]
#plt.hist(statmns.flatten(), bins=onehotsaclust.shape[0], density=True)[2]
sns.kdeplot(statmns.flatten(), bw_adjust=0.2)
#plt.title('')
plt.xlabel('Distance Estimates')
plt.ylabel('Density')
plt.title('Density of Distance Estimates')
plt.savefig('figs/distDens.png')
plt.show()

In [ ]:
clusterDiagnostics(statmns, K=K, labels=confounders[sz:, 0],
                           lo=0, hi=0.0005, step=0.000025, figsize=(8,6))
plt.ylabel('Clustering Accuracy')
plt.title('Accuracy Against Thresholds')
plt.savefig('figs/clustAccs.png')

In [ ]:
plt.figure(figsize=(8,6))
plt.scatter([max(np.mean(i == labels), np.mean(i != labels)) for i in np.array(labelsemsoft).argmax(axis=1)], 
            logliksemsoft)
print(np.mean([max(np.mean(i == labels), np.mean(i != labels)) for i in np.array(labelsemsoft).argmax(axis=1)]))
print(np.median([max(np.mean(i == labels), np.mean(i != labels)) for i in np.array(labelsemsoft).argmax(axis=1)]))
print(np.max([max(np.mean(i == labels), np.mean(i != labels)) for i in np.array(labelsemsoft).argmax(axis=1)]))
plt.ticklabel_format(style='plain',useOffset=False)
plt.xlabel('Accuracy')
plt.ylabel('Log-Likelihood')
plt.title('Randomly Initialized Soft EM Algorithm')
plt.savefig('figs/softEM.png')
plt.show()
plt.hist([max(np.mean(i == labels), np.mean(i != labels)) for i in np.array(labelsemsoft).argmax(axis=1)])
plt.show()

In [ ]:
#plt.figure(figsize=(16,9))
print(np.mean(np.array(clusterAccsTrials),0)[-1])
plt.plot(horizons, 1-np.mean(np.array(clusterAccsTrials),0), label='With Subspaces')
plt.plot(horizons, 1-np.mean(np.array(clusterAccsNoProjsTrials),0), label='Without Subspaces',
        linestyle='--')
plt.ylabel("Clustering Error")
plt.xlabel("Trajectory Length")
plt.title("Clustering Error vs Trajectory Length")
plt.legend()
plt.savefig('figs/clust.png')

In [ ]:
#plt.figure(figsize=(16,9))
plt.plot(horizons, 1-np.mean(np.array(clusterAccsTrials),0), label='With Subspaces')
plt.plot(horizons, 1-np.mean(np.array(clusterAccsNoProjsTrials),0), label='Without Subspaces',
        linestyle='--')
plt.plot(horizons, 1-np.mean(np.array(clusterAccsRandProjsTrials),0), label='Random Subspaces',
        linestyle='dashdot')
plt.ylabel("Clustering Error")
plt.xlabel("Trajectory Length")
plt.title("Clustering Error vs Trajectory Length")
plt.legend()
plt.savefig('figs/clustRand.png')

In [ ]:
np.mean(np.array(clusterAccsTrials),0)

In [ ]:
#plt.figure(figsize=(16,9))
plt.plot(horizons, 1-np.mean(np.array(classAccsTrials),0), label='E-Step')
plt.plot(horizons, 1-np.mean(np.array(classProjAccsTrials),0), label='Classification (Alg. 3)',
        linestyle='--')
plt.ylabel("Classification Error")
plt.xlabel("Trajectory Length")
plt.title("Classification Error vs Trajectory Length")
plt.legend()

In [ ]:
#plt.figure(figsize=(16,9))
'''
plt.plot(horizons, 1-np.median(np.array(hardClustEMAccsTrials),0),
         label='Hard EM Cluster Init')
plt.plot(horizons, 1-np.median(np.array(hardEMAccsTrials),0),
        label='Hard EM')
'''  
print(np.mean(np.array(softClassEMAccsTrials),0)[-1])
plt.plot(horizons, 1-np.mean(np.array(softClassEMAccsTrials),0),
         label='Soft EM Class+Cluster Init')
plt.plot(horizons, 1-np.mean(np.array(softClustEMAccsTrials),0),
         label='Soft EM Cluster Init',
        linestyle='--')
plt.plot(horizons, 1-np.mean(np.array(softEMAccsTrials),0),
        label='Soft EM Random Init',
        linestyle='dashdot')
#plt.plot(horizons, 1-np.mean(np.array(hardClustEMAccsTrials),0),
#         label='Hard EM Cluster Init',
#        linestyle='--')
#plt.plot(horizons, 1-np.mean(np.array(hardEMAccsTrials),0),
#        label='Hard EM Random Init',
#        linestyle='dashdot')
plt.ylabel("Error")
plt.xlabel("Trajectory Length")
plt.title("End-to-End Performance Comparison")
plt.legend()
plt.savefig('figs/emClust.png')

In [ ]:
#plt.figure(figsize=(16,9))
'''
plt.plot(horizons, 1-np.median(np.array(hardClustEMAccsTrials),0),
         label='Hard EM Cluster Init')
plt.plot(horizons, 1-np.median(np.array(hardEMAccsTrials),0),
        label='Hard EM')
'''  
print(np.mean(np.array(softClassEMAccsTrials),0)[-1])
plt.plot(horizons, 1-np.mean(np.array(softClustEMAccsTrials),0),
         label='Soft EM Cluster Init')
plt.plot(horizons, 1-np.mean(np.array(softEMAccsTrials),0),
        label='Soft EM Random Init')
plt.plot(horizons, 1-np.mean(np.array(hardClustEMAccsTrials),0),
         label='Hard EM Cluster Init',
        linestyle='--')
plt.plot(horizons, 1-np.mean(np.array(hardEMAccsTrials),0),
        label='Hard EM Random Init',
        linestyle='--')
plt.ylabel("Error")
plt.xlabel("Trajectory Length")
plt.title("Soft v.s. Hard EM Algorithm")
plt.legend()
plt.savefig('figs/emSoftHard.png')

In [ ]:
from scipy.cluster.hierarchy import linkage

m = 1-statmns #need similarity matrix
# shuffle
perm = np.random.permutation(len(m))
m = m[perm][:, perm]

# reorder
y = m[np.triu_indices(len(m), k=1)]
Z = linkage(y, method='single', optimal_ordering=True)
perm = np.ravel(Z[:, :2]).astype(np.int32)
perm = perm[perm < len(m)]
m = m[perm][:, perm]

#plt.figure(figsize=(16,9))
plt.imshow(1-m)
plt.title('Block Structure of Squared Distances')
plt.savefig('figs/blocks.png')

In [ ]:
vals, vecs = getEig(onehotsa[:sz], onehotsp[:sz], omegaone, omegatwo, 10000, wt=True)
plt.hist((np.sort(vals, axis=-1)**2).mean((0,1)), bins=nStates)
plt.title('Histogram of Mean Ordered Eigenvalue Energy')
plt.ylabel('Count')
plt.xlabel('Eigenvalue Energy')
plt.savefig('figs/eigEnergy.png')
plt.show()

In [ ]:
d_min = np.minimum(getN_sa(dataset[dataset[:,:,2]==0], nStates, nActions, burnin=0, reshape=False)/len(dataset[dataset[:,:,2]==0]),
           getN_sa(dataset[dataset[:,:,2]==1], nStates, nActions, burnin=0, reshape=False)/len(dataset[dataset[:,:,2]==1]))

In [ ]:
plt.scatter(d_min.flatten(), np.linalg.norm((P[0])-(P[1]), axis=-1).flatten())
plt.xlabel('Estimated Occupancy Measures')
plt.ylabel('True Model Separation')
plt.title('Model Separation v.s. Occupancy Measures')
plt.savefig('figs/trueModelSep.png')

In [ ]:
plt.scatter(d_min.flatten(), np.linalg.norm((modelEstims[-1][0])-(modelEstims[-1][1]), axis=-1).flatten())
plt.xlabel('Estimated Occupancy Measures', fontsize=13)
plt.ylabel('Clustering-Estimated Model Separation', fontsize=13)
plt.title('Model Separation v.s. Occupancy Measures')
plt.savefig('figs/clustModelSep.png')